In [6]:
import os
import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors
import pandas as pd
from googleapiclient.discovery import build
from youtube_transcript_api import YouTubeTranscriptApi  

**Retriving Video IDs for respective keyword according to provided timeline**

In [ ]:
scopes = ["https://www.googleapis.com/auth/youtube.force-ssl"]


def main():
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"
    api_service_name = "youtube"
    api_version = "v3"
    client_secrets_file = "cs.json"
    flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(client_secrets_file, scopes)
    credentials = flow.run_console()
    youtube = googleapiclient.discovery.build(
    api_service_name, api_version, credentials=credentials)
    request = youtube.search().list(part='snippet',maxResults=50,q="Metastatic CRPC",publishedAfter='2022-08-11T00:00:00Z',publishedBefore='2022-08-18T00:00:00Z',order='date')
    response = request.execute()
    
    video_id = []
    
    for i in range(len(response['items'])):
        video_id.append(response['items'][i]['id']['videoId'])
        
    next_page_token = response.get('nextPageToken')
    more_pages = True
    
    while more_pages:
        if next_page_token is None:
            more_pages = False
        else:
            request = youtube.search().list(part='snippet',maxResults=50,q="Metastatic CRPC",pageToken = next_page_token,publishedAfter='2022-08-11T00:00:00Z',publishedBefore='2022-08-18T00:00:00Z',order='date')
            response = request.execute()
    
            for i in range(len(response['items'])):
              try:
                video_id.append(response['items'][i]['id']['videoId'])
              except:
                pass
            
            next_page_token = response.get('nextPageToken')
    
    channel_data = pd.DataFrame(video_id)    
    return channel_data

video_id = main()
try:
  vid = video_id[0].tolist()
  vid
except:
  print("No videos Available")




**Checking Availability of transcripts**


In [ ]:
try:
  final=[]
  for i in vid:
    try:
      transcript_list = YouTubeTranscriptApi.list_transcripts(i)
      for transcript in transcript_list:
        final.append({i:transcript.is_generated})
    except:
      final.append({i:False})  
  d = {k:v for x in final for k,v in x.items()}
  d
except:
  print("No videos hence no transcripts") 


**Retriving the English transcripts**

In [ ]:
try:
  output = []
  for key, value in d.items():
    if value==True:
      try:
        srt = YouTubeTranscriptApi.get_transcript(key,languages=['en'])
        df1 = pd.DataFrame(srt)
        output.append({key:' '.join(df1['text'])})
      except:
        output.append({key:"Not Available in English"})
    else:
        output.append({key:'No transcript'})
except:
  print("No videos hence no transcripts")

        

In [ ]:
output
